In [2]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt 
import seaborn as sns
from bs4 import BeautifulSoup
import os
from time import sleep
from urllib.request import Request, urlopen
from urllib.error import URLError, HTTPError
import random 

In [3]:
import spacy
nlp = spacy.load("pt_core_news_sm")

In [4]:
folha_1_url = "../base_aplicacao_resultado/rotulado_tratado_links_folha_sem_rotulo_resgatados.csv"

In [5]:
folha = pd.read_csv(folha_1_url, index_col = [0])
print("Quantidade dados folha: ", folha.shape) 

Quantidade dados folha:  (3318, 6)


In [6]:
print(folha[folha.link.str.contains("/opiniao/")].shape)
folha[folha.link.str.contains("/opiniao/")].head()

(175, 6)


,ano,link,paragrafos_resgatados,paragrafos_tratados_sw,paragrafos_tratados_sw_vetorizados,metaforico_pred
133,2001,http://www1.folha.uol.com.br/fsp/opiniao/fz210...,"['O pano de fundo era, evidentemente, a esquiz...",pano fundo evidentemente esquizofrenia alemanh...,[[-0.64273801 0.59776301 0.91133999 0.54527...,1
137,2001,http://www1.folha.uol.com.br/fsp/opiniao/fz011...,['Esta semana certamente vai impor aos brasile...,semana impor brasileiros noticiário esquizofrê...,[[ 3.65856902e+00 -1.15379974e-01 4.06036301e...,1
138,2001,http://www1.folha.uol.com.br/fsp/opiniao/fz011...,['Esta semana certamente vai impor aos brasile...,esquizofrenia caso fica conta situações inusit...,[[ 1.07493307e+00 1.96975901e+00 1.87077704e...,1
141,2001,http://www1.folha.uol.com.br/fsp/opiniao/fz301...,"[', pág. A2) descontextualiza minhas opiniões ...",descontextualiza opiniões mercosul defendido m...,[[ 5.59070402e+00 6.42139397e-02 1.64989005e...,1
163,2001,http://www1.folha.uol.com.br/fsp/opiniao/fz121...,['Estima-se que 1 em cada 4 pessoas deve apres...,pessoas apresentar transtorno mental longo vid...,[[ 1.23243002e-01 2.07713400e+00 3.29981903e...,0


In [7]:
folha[folha.link.str.contains('paineldoleitor')].shape

(33, 6)

In [8]:
folha.loc[(folha.link.str.contains("paineldoleitor|painel-do-leitor")), "painel_do_leitor"] = "sim"
folha["painel_do_leitor"].fillna("nao", inplace = True)

In [9]:
folha.painel_do_leitor.value_counts()

nao    3281
sim      37
Name: painel_do_leitor, dtype: int64

In [10]:
folha[(folha.link.str.contains("/opiniao/"))]

,ano,link,paragrafos_resgatados,paragrafos_tratados_sw,paragrafos_tratados_sw_vetorizados,metaforico_pred,painel_do_leitor
133,2001,http://www1.folha.uol.com.br/fsp/opiniao/fz210...,"['O pano de fundo era, evidentemente, a esquiz...",pano fundo evidentemente esquizofrenia alemanh...,[[-0.64273801 0.59776301 0.91133999 0.54527...,1,nao
137,2001,http://www1.folha.uol.com.br/fsp/opiniao/fz011...,['Esta semana certamente vai impor aos brasile...,semana impor brasileiros noticiário esquizofrê...,[[ 3.65856902e+00 -1.15379974e-01 4.06036301e...,1,nao
138,2001,http://www1.folha.uol.com.br/fsp/opiniao/fz011...,['Esta semana certamente vai impor aos brasile...,esquizofrenia caso fica conta situações inusit...,[[ 1.07493307e+00 1.96975901e+00 1.87077704e...,1,nao
141,2001,http://www1.folha.uol.com.br/fsp/opiniao/fz301...,"[', pág. A2) descontextualiza minhas opiniões ...",descontextualiza opiniões mercosul defendido m...,[[ 5.59070402e+00 6.42139397e-02 1.64989005e...,1,nao
163,2001,http://www1.folha.uol.com.br/fsp/opiniao/fz121...,['Estima-se que 1 em cada 4 pessoas deve apres...,pessoas apresentar transtorno mental longo vid...,[[ 1.23243002e-01 2.07713400e+00 3.29981903e...,0,nao
...,...,...,...,...,...,...,...
3229,2009,http://www1.folha.uol.com.br/fsp/opiniao/fz170...,['Solidários com a angústia e a dor de quem te...,solidários angústia dor filhos esquizofrênicos...,[[-1.93052957e-01 -6.90079996e-01 5.81792978e...,1,nao
3253,2014,http://www1.folha.uol.com.br/opiniao/2014/05/1...,"['Suspeito que o termo dívida interna, de memó...",suspeito termo dívida interna memória econômic...,[[ 5.31578093 1.83855503 4.74310406 0.54274...,1,nao
3254,2014,http://www1.folha.uol.com.br/fsp/opiniao/16805...,"['Suspeito que o termo dívida interna, de memó...",suspeito termo dívida interna memória econômic...,[[ 5.51212393 1.93645803 5.08084505 0.57992...,1,nao
3256,2014,http://www1.folha.uol.com.br/opiniao/2014/02/1...,"['No Brasil de valores esquizofrênicos, pode-s...",brasil valores esquizofrênicos matar cidadão s...,[[ 8.62740165e-02 2.44449005e-01 2.75677981e...,1,nao


In [11]:
dt = folha[(folha.link.str.contains("/opiniao/")) & (folha.painel_do_leitor == "nao")]
dt.head()

,ano,link,paragrafos_resgatados,paragrafos_tratados_sw,paragrafos_tratados_sw_vetorizados,metaforico_pred,painel_do_leitor
133,2001,http://www1.folha.uol.com.br/fsp/opiniao/fz210...,"['O pano de fundo era, evidentemente, a esquiz...",pano fundo evidentemente esquizofrenia alemanh...,[[-0.64273801 0.59776301 0.91133999 0.54527...,1,nao
137,2001,http://www1.folha.uol.com.br/fsp/opiniao/fz011...,['Esta semana certamente vai impor aos brasile...,semana impor brasileiros noticiário esquizofrê...,[[ 3.65856902e+00 -1.15379974e-01 4.06036301e...,1,nao
138,2001,http://www1.folha.uol.com.br/fsp/opiniao/fz011...,['Esta semana certamente vai impor aos brasile...,esquizofrenia caso fica conta situações inusit...,[[ 1.07493307e+00 1.96975901e+00 1.87077704e...,1,nao
141,2001,http://www1.folha.uol.com.br/fsp/opiniao/fz301...,"[', pág. A2) descontextualiza minhas opiniões ...",descontextualiza opiniões mercosul defendido m...,[[ 5.59070402e+00 6.42139397e-02 1.64989005e...,1,nao
163,2001,http://www1.folha.uol.com.br/fsp/opiniao/fz121...,['Estima-se que 1 em cada 4 pessoas deve apres...,pessoas apresentar transtorno mental longo vid...,[[ 1.23243002e-01 2.07713400e+00 3.29981903e...,0,nao


In [12]:
dt.shape

(171, 7)

In [13]:
def limpa_paragrafos(link, tamanho):
    
    headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.77 Safari/537.36"}
    try:
        req = Request(link, headers = headers)
        response = urlopen(req)
       
    except HTTPError as e: # erro de segurança do site, pode ser necessário usar user-Agent
        print(e.status, e.reason)

    except URLError as e:
        print(e.reason)
    
    html = response.read()
    soup = BeautifulSoup(html)

    for script in soup(["script", "style"]):
        script.decompose()

    lista = []
    for i in soup.stripped_strings:
        lista.append(i)

    # miolo = []
    # for i in lista:
    #     if len(i) > tamanho:
    #         miolo.append(i.replace('\r\n',''))

    texto = " ".join(lista).replace("\n", "").replace("\r", "")
    
    return texto

In [14]:
# url = dt[dt.link.str.contains("fz1504200910")].link.values[0]
url = "http://www1.folha.uol.com.br/fsp/opiniao/fz0110200110.htm"

In [15]:
url

'http://www1.folha.uol.com.br/fsp/opiniao/fz0110200110.htm'

In [16]:
texto = limpa_paragrafos(url, 70)

In [17]:
texto

'Folha de S.Paulo - Sávio Souza Cruz: Terrorismo partidário - 01/10/2001 São Paulo, segunda-feira, 01 de outubro de 2001 Texto Anterior | Próximo Texto | Índice TENDÊNCIAS/DEBATES Terrorismo partidário SÁVIO SOUZA CRUZ Esta semana certamente vai impor aos brasileiros um noticiário próximo do esquizofrênico, em que o leitor atento será levado a desconfiar dos jornais nas referências que forem feitas às siglas partidárias dos políticos. A culpa não será dos jornais, mas do calendário eleitoral, que, ao definir o dia 5, sexta-feira, como prazo fatal para filiação partidária dos candidatos ao próximo pleito, desencadeará uma verdadeira ciranda, com mudanças e trocas de cadeiras nem sempre explicáveis. A esquizofrenia, no caso, fica por conta de algumas situações inusitadas e incompreensíveis, só viabilizadas pela fragilidade de nossa legislação, que não cobra coerência dos atores políticos nem oferece consistência às agremiações partidárias, permanentemente vulneráveis, tal qual torres de 

In [18]:
"PAINEL DO LEITOR" in texto

False

In [19]:
dt.head()

,ano,link,paragrafos_resgatados,paragrafos_tratados_sw,paragrafos_tratados_sw_vetorizados,metaforico_pred,painel_do_leitor
133,2001,http://www1.folha.uol.com.br/fsp/opiniao/fz210...,"['O pano de fundo era, evidentemente, a esquiz...",pano fundo evidentemente esquizofrenia alemanh...,[[-0.64273801 0.59776301 0.91133999 0.54527...,1,nao
137,2001,http://www1.folha.uol.com.br/fsp/opiniao/fz011...,['Esta semana certamente vai impor aos brasile...,semana impor brasileiros noticiário esquizofrê...,[[ 3.65856902e+00 -1.15379974e-01 4.06036301e...,1,nao
138,2001,http://www1.folha.uol.com.br/fsp/opiniao/fz011...,['Esta semana certamente vai impor aos brasile...,esquizofrenia caso fica conta situações inusit...,[[ 1.07493307e+00 1.96975901e+00 1.87077704e...,1,nao
141,2001,http://www1.folha.uol.com.br/fsp/opiniao/fz301...,"[', pág. A2) descontextualiza minhas opiniões ...",descontextualiza opiniões mercosul defendido m...,[[ 5.59070402e+00 6.42139397e-02 1.64989005e...,1,nao
163,2001,http://www1.folha.uol.com.br/fsp/opiniao/fz121...,['Estima-se que 1 em cada 4 pessoas deve apres...,pessoas apresentar transtorno mental longo vid...,[[ 1.23243002e-01 2.07713400e+00 3.29981903e...,0,nao


In [20]:
for url in dt.link.unique():
    texto = limpa_paragrafos(url, 70)

    if ("PAINEL DO LEITOR" in texto):
        dt.loc[(dt.link == url), "opiniao_painel_do_leitor"] = "sim"
    
    n = random.randint(0, 5)

    sleep(n)

c:\Users\r-omd\.conda\envs\tf-venv\lib\site-packages\pandas\core\indexing.py:1684: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = infer_fill_value(value)
c:\Users\r-omd\.conda\envs\tf-venv\lib\site-packages\pandas\core\indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


KeyboardInterrupt: 

In [ ]:
dt.head()

,ano,link,paragrafos_resgatados,paragrafos_tratados_sw,paragrafos_tratados_sw_vetorizados,metaforico_pred,painel_do_leitor,opiniao_painel_do_leitor
133,2001,http://www1.folha.uol.com.br/fsp/opiniao/fz210...,"['O pano de fundo era, evidentemente, a esquiz...",pano fundo evidentemente esquizofrenia alemanh...,[[-0.64273801 0.59776301 0.91133999 0.54527...,1,nao,NaN
137,2001,http://www1.folha.uol.com.br/fsp/opiniao/fz011...,['Esta semana certamente vai impor aos brasile...,semana impor brasileiros noticiário esquizofrê...,[[ 3.65856902e+00 -1.15379974e-01 4.06036301e...,1,nao,NaN
138,2001,http://www1.folha.uol.com.br/fsp/opiniao/fz011...,['Esta semana certamente vai impor aos brasile...,esquizofrenia caso fica conta situações inusit...,[[ 1.07493307e+00 1.96975901e+00 1.87077704e...,1,nao,NaN
141,2001,http://www1.folha.uol.com.br/fsp/opiniao/fz301...,"[', pág. A2) descontextualiza minhas opiniões ...",descontextualiza opiniões mercosul defendido m...,[[ 5.59070402e+00 6.42139397e-02 1.64989005e...,1,nao,sim
163,2001,http://www1.folha.uol.com.br/fsp/opiniao/fz121...,['Estima-se que 1 em cada 4 pessoas deve apres...,pessoas apresentar transtorno mental longo vid...,[[ 1.23243002e-01 2.07713400e+00 3.29981903e...,0,nao,NaN


In [ ]:
dt.link.values[1]

'http://www1.folha.uol.com.br/fsp/opiniao/fz0110200110.htm'

In [ ]:
dt.opiniao_painel_do_leitor.fillna("nao", inplace = True)

C:\Users\r-omd\AppData\Local\Temp\ipykernel_81156\2544169676.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dt.opiniao_painel_do_leitor.fillna("nao", inplace = True)


In [ ]:
dt.head()

,ano,link,paragrafos_resgatados,paragrafos_tratados_sw,paragrafos_tratados_sw_vetorizados,metaforico_pred,painel_do_leitor,opiniao_painel_do_leitor
133,2001,http://www1.folha.uol.com.br/fsp/opiniao/fz210...,"['O pano de fundo era, evidentemente, a esquiz...",pano fundo evidentemente esquizofrenia alemanh...,[[-0.64273801 0.59776301 0.91133999 0.54527...,1,nao,nao
137,2001,http://www1.folha.uol.com.br/fsp/opiniao/fz011...,['Esta semana certamente vai impor aos brasile...,semana impor brasileiros noticiário esquizofrê...,[[ 3.65856902e+00 -1.15379974e-01 4.06036301e...,1,nao,nao
138,2001,http://www1.folha.uol.com.br/fsp/opiniao/fz011...,['Esta semana certamente vai impor aos brasile...,esquizofrenia caso fica conta situações inusit...,[[ 1.07493307e+00 1.96975901e+00 1.87077704e...,1,nao,nao
141,2001,http://www1.folha.uol.com.br/fsp/opiniao/fz301...,"[', pág. A2) descontextualiza minhas opiniões ...",descontextualiza opiniões mercosul defendido m...,[[ 5.59070402e+00 6.42139397e-02 1.64989005e...,1,nao,sim
163,2001,http://www1.folha.uol.com.br/fsp/opiniao/fz121...,['Estima-se que 1 em cada 4 pessoas deve apres...,pessoas apresentar transtorno mental longo vid...,[[ 1.23243002e-01 2.07713400e+00 3.29981903e...,0,nao,nao


In [ ]:
dt.query("opiniao_painel_do_leitor == 'sim'")

,ano,link,paragrafos_resgatados,paragrafos_tratados_sw,paragrafos_tratados_sw_vetorizados,metaforico_pred,painel_do_leitor,opiniao_painel_do_leitor
141,2001,http://www1.folha.uol.com.br/fsp/opiniao/fz301...,"[', pág. A2) descontextualiza minhas opiniões ...",descontextualiza opiniões mercosul defendido m...,[[ 5.59070402e+00 6.42139397e-02 1.64989005e...,1,nao,sim
275,2002,http://www1.folha.uol.com.br/fsp/opiniao/fz240...,"['""É da mais alta seriedade o alerta do artigo...",alta seriedade alerta artigo pacto salvar paul...,[[ 2.82425000e+00 7.96742020e-01 2.66164095e...,1,nao,sim
327,2002,http://www1.folha.uol.com.br/fsp/opiniao/fz111...,"['""O crime na família germânico-paulistana ext...",crime família extrapolou pensávamos inatingíve...,[[ 1.75356202e+00 1.39063099e+00 -1.34848402e...,1,nao,sim
360,2003,http://www1.folha.uol.com.br/fsp/opiniao/fz130...,"['""Como mestrando em sociologia na Universidad...",mestrando sociologia universidade federal goiá...,[[ 4.72369000e+00 6.22676048e-01 -3.05661955e...,1,nao,sim
820,2008,http://www1.folha.uol.com.br/fsp/opiniao/fz211...,"['""Fico abismado com a esquizofrenia na políti...",fico abismado esquizofrenia política brasileir...,[[ 1.00880001e+00 -7.33095024e-01 6.72786975e...,1,nao,sim
935,2009,http://www1.folha.uol.com.br/fsp/opiniao/fz270...,['Digo como mãe de um filho sofrendo de esquiz...,digo mãe filho sofrendo esquizofrenia durante ...,[[ 2.93497904e+00 1.48484005e+00 1.43815201e...,0,nao,sim
1010,2009,http://www1.folha.uol.com.br/fsp/opiniao/fz220...,"['""Gostaria de parabenizar o companheiro Jorge...",gostaria parabenizar companheiro jorge cândido...,[[ 9.53114011e-01 -2.85746001e-01 1.66462401e...,0,nao,sim
1020,2009,http://www1.folha.uol.com.br/fsp/opiniao/fz150...,"['""Expresso aqui minha concordância em relação...",expresso concordância artigo ferreira gullar p...,[[ 1.70981500e+00 9.67188004e-01 4.59812986e...,0,nao,sim
1021,2009,http://www1.folha.uol.com.br/fsp/opiniao/fz140...,"['""Excelente a crítica de Gullar. Como irmão d...",excelente crítica gullar irmão esquizofrênico ...,[[ 8.23789021e-01 -1.21747974e-01 1.70347201e...,0,nao,sim
1022,2009,http://www1.folha.uol.com.br/fsp/opiniao/fz140...,"['""Excelente a crítica de Gullar. Como irmão d...",periódicos indicam diagnóstico gullar informa ...,[[ 1.34345400e+00 -2.86577989e-01 1.56544598e...,0,nao,sim


In [ ]:
dt = dt.sort_values("ano")

In [ ]:
folha.shape

(3318, 7)

In [ ]:
folha_painel = folha.merge(dt[["link", "opiniao_painel_do_leitor"]], on = "link", how = "left").drop_duplicates()
folha_painel.shape

(3318, 8)

In [ ]:
folha_painel.opiniao_painel_do_leitor.value_counts()

nao    146
sim     25
Name: opiniao_painel_do_leitor, dtype: int64

In [ ]:
folha_painel.opiniao_painel_do_leitor.fillna("nao", inplace = True)

In [ ]:
folha_painel.loc[(folha_painel.link.str.contains("/opiniao/")) & (folha_painel.opiniao_painel_do_leitor == "nao"), "opiniao"] = "sim"

In [ ]:
folha_painel.opiniao.fillna("nao", inplace = True)

In [ ]:
folha_painel.loc[(folha_painel.painel_do_leitor == "sim") | (folha_painel.opiniao_painel_do_leitor == "sim"), "painel_leitor"] = "sim"
folha_painel.painel_leitor.fillna("nao", inplace = True)

In [ ]:
folha_painel.painel_do_leitor.value_counts()

nao    3281
sim      37
Name: painel_do_leitor, dtype: int64

In [ ]:
# folha_painel.to_csv("base_folha_analise.csv ", index = False)

In [ ]:
folha_painel

,ano,link,paragrafos_resgatados,paragrafos_tratados_sw,paragrafos_tratados_sw_vetorizados,metaforico_pred,painel_do_leitor,opiniao_painel_do_leitor,opiniao,painel_leitor
0,2001,http://www1.folha.uol.com.br/fsp/ilustrad/fq04...,['É uma idiotice. Mas é o preço do sucesso dos...,idiotice preço sucesso antidepressivos transfo...,[[ 1.52341600e+00 -7.87311998e-01 -1.90507002e...,0,nao,nao,nao,nao
1,2001,http://www1.folha.uol.com.br/folha/mundo/ult94...,"['Os transtornos psíquicos, inclusive as manif...",transtornos psíquicos manifestações depressiva...,[[-3.08092985e-01 8.02221035e-01 1.68601102e...,0,nao,nao,nao,nao
2,2001,http://www1.folha.uol.com.br/fsp/ilustrad/fq10...,['O artista plástico Ubirajara Ferreira Braga ...,artista plástico ubirajara ferreira braga come...,[[ 3.43677802e+00 1.09151603e+00 3.13246397e...,0,nao,nao,nao,nao
3,2001,http://www1.folha.uol.com.br/folha/ciencia/ult...,['Folha Online - Ciência - Israelenses descobr...,folha online ciência israelenses descobrem gen...,[[ 2.26201983e-01 5.72138001e-01 1.13995994e...,0,nao,nao,nao,nao
4,2001,http://www1.folha.uol.com.br/folha/ciencia/ult...,['Folha Online - Ciência - Israelenses descobr...,pesquisadores israelenses descobriu gene servi...,[[ 1.61639995e-01 1.52443696e+00 7.48734023e...,0,nao,nao,nao,nao
...,...,...,...,...,...,...,...,...,...,...
3437,2017,https://www1.folha.uol.com.br/mercado/2017/08/...,"['O anúncio agrada às empresas do setor, disse...",anúncio agrada empresas setor disse presidente...,[[ 1.27768403e+00 -5.16145026e-01 2.43840205e...,1,nao,nao,nao,nao
3438,2018,https://f5.folha.uol.com.br/viva-bem/2018/06/a...,['Uma pesquisa publicada em 1998 no Journal of...,pesquisa publicada journal of child neurology ...,[[ 1.50322202e+00 2.85787299e+00 3.16622208e...,0,nao,nao,nao,nao
3439,2019,https://www1.folha.uol.com.br/poder/2019/10/ev...,"['""O advogado e ex-ministro Admar e\xa0a advog...",advogado admar advogada karina kufa trabalhand...,[[ 1.43166601e+00 -3.75220021e-01 1.10605499e...,1,nao,nao,nao,nao
3440,2020,https://www1.folha.uol.com.br/opiniao/2020/01/...,['Tanta pressa gera reações esquizofrênicas. V...,pressa gera reações esquizofrênicas vejam busc...,[[ 2.43236 -1.50618503 4.50067598 -2.61170...,1,nao,nao,sim,nao


In [ ]:
folha_painel.to_csv("../analises_bases_unidas/bases/base_folha_app_analise.csv", index = False)